In [1]:
import numpy as np

# Автоколл

Задача 1. Рассмотрим структурный продукт типа автоколл на бездивидендные акции с динамикой  $S_t \sim GBM(\mu,\sigma)$, $S(0) = 100$ и волатильностью $25\%$, безрисковая ставка $0\%$ и следующей функцией выплаты:

\begin{equation*}
%X(\omega) =
 \begin{cases}
%   1 &\text{se $\omega\in A$}\\
%   1250 &\text{se $\omega \in A^c$}
    (1 + c),  & \text{если по истечении первого года } S(1) > S(0),\\
              & \text{продукт прекращает свое действие},\\
    (1 + 2c), & \text{если через два года } S(2) > S(0),\\
              & \text{продукт прекращает свое действие};\\
    \max\left(1 + 3c, \frac{S(3)}{S(0)}\right), & \text{если через три года }
                                                S(3) > 0.7 \times S(0), \\
                                                & \text{продукт прекращает свое действие};\\
    \frac{S(3)}{S(0)},& \text{во остальных случаях}.
   \end{cases}
\end{equation*}


Применяя моделирование Монте-Карло определите численное значение купона $c$, при котором выплата составит $\frac{S(3)}{S(0)}$.

In [2]:
%%time

r = 0
q = 0
mu     = r - q
sigma  = 0.25
S0     = 100
nPaths = 2_000_000


simuls = []
for _ in range(nPaths):
    S1 = S0 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    S2 = S1 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    S3 = S2 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    simuls.append([S1, S2, S3])

coupon = 0.0765 # определить программно
payoff = []

for simul in simuls:
    if simul[0]>S0:
        payoff.append(1+coupon)
    elif simul[1]>S0:
        payoff.append(1+2*coupon)
    elif simul[2]>0.7*S0:
        payoff.append(max(1+3*coupon, simul[2]/S0))
    else:
        payoff.append(simul[2]/S0)
        
print(np.mean(payoff))

1.000819009712145
CPU times: user 15.5 s, sys: 239 ms, total: 15.7 s
Wall time: 15.7 s


In [168]:
%%time
np_simuls = np.asarray(simuls)
np_simuls_mean = np_simuls.mean(axis=0)
print('Mean S(3)/S(0) ~ ', np_simuls_mean[2]/S0)

def coupon_dependent_payoff(coupon):   
    payoff = []
    for simul in simuls:
        if simul[0] > S0:
            payoff.append(1 + coupon)
        elif simul[1] > S0:
            payoff.append(1 + 2 * coupon)
        elif simul[2] > 0.7 * S0:
            payoff.append(max(1 + 3 * coupon, simul[2] / S0))
        else:
            payoff.append(simul[2]/S0)
    return np.mean(payoff)

def calc_derivative(func, x, dx=1e-2):
    return (func(x + dx) - func(x - dx)) / 2 / dx

def mse_loss(coupon):
    return (coupon_dependent_payoff(coupon) - np_simuls_mean[2]/S0)**2

coupon = 0
alpha = 0.25
i = 0
mse_loss_v = 1
while mse_loss_v > 1e-8:
    mse_loss_v = mse_loss(coupon)
    i += 1
    deriv = calc_derivative(mse_loss, coupon)
    coupon -= alpha * deriv
    # if i % 5 == 0:
    print('Iteration = ', i, ', coupon = ', coupon, ' mse_loss = ', mse_loss_v)
print('Numerically calculated coupon = ', coupon)

Mean S(3)/S(0) ~  0.9994269035382068
Iteration =  1 , coupon =  0.05311276633328917  mse_loss =  0.008547935781005015
Iteration =  2 , coupon =  0.07078383014223595  mse_loss =  0.000801386057443831
Iteration =  3 , coupon =  0.07457549648936236  mse_loss =  3.618968282678929e-05
Iteration =  4 , coupon =  0.07530702497977455  mse_loss =  1.419757421015917e-06
Iteration =  5 , coupon =  0.07544534242259043  mse_loss =  6.724165315778523e-08
Iteration =  6 , coupon =  0.07547139741793459  mse_loss =  6.8883946167220155e-09
Numerically calculated coupon =  0.07547139741793459
CPU times: user 11.5 s, sys: 531 ms, total: 12 s
Wall time: 12 s


Определите значения |купона| барьера таким образом, чтобы продукт был исполнен в

а) первый год,

б) второй год,

в) третий год.

Приведите графики функций выплат по продукту для всех вариантов.

Сравните функции выплат структурного продукта типа автоколл с ванильным опционом. Приведите иллюстративный график и сделайте выводы.


In [4]:
%%time
np_simuls = np.asarray(simuls)

barrier_95q = np.quantile(np_simuls[:,0], q=0.95)
print('Barrier level to execute the coupon in the 1st year = ', barrier_95q)

nPaths = 100_000


simuls = []
for _ in range(nPaths):
    S1 = S0 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    S2 = S1 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    S3 = S2 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    simuls.append([S1, S2, S3])

b1, b2, b3 = 0, 0, 0
for simul in simuls:
    if simul[0]>S0:
        b1 += 1
    elif simul[1]>S0:
        b2 += 1
    elif simul[2]>0.7*S0:
        b3 += 1
    else:
        payoff.append(simul[2]/S0)

CPU times: user 668 ms, sys: 75.2 ms, total: 743 ms
Wall time: 753 ms


In [7]:
b = 100

In [8]:
nPaths = 100_000
simuls = []
for _ in range(nPaths):
    S1 = S0 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    S2 = S1 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    S3 = S2 * np.exp(mu - 0.5 * sigma**2 + sigma * np.random.normal())
    simuls.append([S1, S2, S3])

b1, b2, b3 = 0, 0, 0
for simul in simuls:
    if simul[0] > b:
        b1 += 1
    elif simul[1] > b:
        b2 += 1
    elif simul[2] > 0.7 * b:
        b3 += 1
    else:
        pass

Задача 2. Предположим, что динамика базового актива описывается геометрическим броуновским движением $S_t \sim GBM(\mu,\sigma)$.

а) Вычислите аналитически вероятность срабатывания автоотзыва $\mathbb{P}(\tau = t_k)=p_k$ для продукта с характеристиками $S(0)=1$, $\mu=5\%$, $\sigma=25\%$,
$t_k=\frac{6+(k-1)}{12}$, $k=1,2, \ldots, 6$.

б) Вычислите вероятность того, что продукт достигнет «конца» своего срока действия:

$(1 − 𝑝_1)(1 − 𝑝_2)(1 − 𝑝_3) … (1 − 𝑝_{𝑛−1})$,

где $𝑝_k$ -- вероятность досрочного погашения в дату проверки условий $𝑡_k$.

Выполните Монте-Карло моделирование и сравните результаты.

Ожидаемое решение для пункта (а): $[45.45, 6.29, 4.1,3.07, 2.43, 1.99]$.

в) Постройте график зависимости вероятности срабатывания автоотзыва от времени.

г) Оцените приведенную стоимость структурного продукта, NVP.

In [217]:
# analytical solution
# using analytical formula

# probability of execution
def execution_proba(S0, mu, sigma, timeline):
    # build covariate matrix
    cov_matrix = np.eye(len(timeline))
    for i, t1 in enumerate(timeline):
        for j, t2 in enumerate(timeline):
            if i != j:
                cov_matrix[i, j] = min(t1, t2)
                cov_matrix[j, i] = cov_matrix[i, j]
    return cov_matrix

In [218]:
timeline = np.linspace(0, 1/2, 6)
execution_proba(0, 0, 0, timeline)

array([[1. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 1. , 0.1, 0.1, 0.1, 0.1],
       [0. , 0.1, 1. , 0.2, 0.2, 0.2],
       [0. , 0.1, 0.2, 1. , 0.3, 0.3],
       [0. , 0.1, 0.2, 0.3, 1. , 0.4],
       [0. , 0.1, 0.2, 0.3, 0.4, 1. ]])

In [214]:
timeline

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5])